# Modifying the `Network` object
`GeNet` supports some simple modifications like adding, reindexing and removing nodes and links and some involved modifications like changing the data stored under nodes or links. All of these changes get recorded in `n.change_log`.

In [1]:
# read example network
from genet import read_matsim
import os

path_to_matsim_network = '../example_data/pt2matsim_network'

network = os.path.join(path_to_matsim_network, 'network.xml')
schedule = os.path.join(path_to_matsim_network, 'schedule.xml')
vehicles = os.path.join(path_to_matsim_network, 'vehicles.xml')
n = read_matsim(
    path_to_network=network, 
    epsg='epsg:27700', 
    path_to_schedule=schedule, 
    path_to_vehicles=vehicles
)
# you don't need to read the vehicles file, but doing so ensures all vehicles
# in the schedule are of the expected type and the definition of the vehicle
# is preserved
n.print()

Graph info: Name: 
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 
Schedule info: Schedule:
Number of services: 9
Number of routes: 68
Number of stops: 118


## Adding nodes/links

In [2]:
n.add_link(link_id='proposed_index', u='4356572310', v='5811263955')

2022-06-20 17:46:26,847 - Added Link with index proposed_index, from node:4356572310 to node:5811263955, under multi-index:1, and data={'from': '4356572310', 'to': '5811263955', 'id': 'proposed_index'}


'proposed_index'

In [3]:
n.add_node(node='proposed_index', attribs={'data':'some_data'})

2022-06-20 17:46:26,877 - Added Node with index `proposed_index` and data={'data': 'some_data'}


'proposed_index'

The index passed is only a proposition. If a node or link under this link exists, a new, unique index will be generated.

In [4]:
actual_link_id_added = n.add_link(link_id='proposed_index', u='4356572310', v='5811263955')

2022-06-20 17:46:26,894 - Generated 1 link ids.
2022-06-20 17:46:26,901 - Generated link id 0.
2022-06-20 17:46:26,903 - `proposed_index` already exists. Generated a new unique_index: `0`
2022-06-20 17:46:26,910 - Added Link with index 0, from node:4356572310 to node:5811263955, under multi-index:2, and data={'from': '4356572310', 'to': '5811263955', 'id': '0'}


## Reindexing

To reindex a node or link:

In [5]:
n.reindex_node('proposed_index', 'another_index')

2022-06-20 17:46:26,974 - Changed Link attributes for 0 links
2022-06-20 17:46:27,012 - Changed Link attributes for 0 links
2022-06-20 17:46:27,018 - Changed Node attributes under index: proposed_index
2022-06-20 17:46:27,171 - Changed Node index from proposed_index to another_index


In [6]:
n.reindex_link('proposed_index', 'another_index')

2022-06-20 17:46:27,195 - Changed Link attributes under index: proposed_index
2022-06-20 17:46:27,199 - Changed Link index from proposed_index to another_index


## Removing nodes/links

To remove a link or node:

In [7]:
n.remove_links(['another_index', actual_link_id_added])

2022-06-20 17:46:27,249 - Removed 2 links


In [8]:
n.remove_node('another_index')

2022-06-20 17:46:27,284 - Removed Node under index: another_index


## Modifying data stored on nodes or edges:

Let's say you have extracted `genet.Network` link ids of interest (See Section on Using Network - Accessing Data) and now you want to make changes to the network. Let's make changes to the nested OSM data stored on the links. We will replace the highway tags from `'primary'` to `'SOMETHING'`.

In [9]:
from genet import graph_operations

links = n.extract_links_on_edge_attributes(
    conditions= {'attributes': {'osm:way:highway': 'primary'}},
)

links[:5]

['1007', '1008', '1023', '1024', '103']

In [10]:
n.link(links[0])

{'id': '1007',
 'from': '4356572310',
 'to': '5811263955',
 'freespeed': 22.22222222222222,
 'capacity': 3000.0,
 'permlanes': 2.0,
 'oneway': '1',
 'modes': {'car'},
 's2_from': 5221390723045407809,
 's2_to': 5221390723040504387,
 'attributes': {'osm:way:highway': 'primary',
  'osm:way:id': 589660342.0,
  'osm:way:lanes': '2',
  'osm:way:name': 'Shaftesbury Avenue',
  'osm:way:oneway': 'yes'},
 'length': 13.941905154249884}

We create a dictionary which maps same changes to all links in the list using:

```python
    {link_id: {'attributes': {'osm:way:highway': 'SOMETHING'}} for link_id in links}
```

In [11]:
n.apply_attributes_to_links({link_id: {'attributes': {'osm:way:highway': 'SOMETHING'}} for link_id in links})

2022-06-20 17:46:27,498 - Changed Link attributes for 619 links


In [12]:
n.link(links[0])

{'id': '1007',
 'from': '4356572310',
 'to': '5811263955',
 'freespeed': 22.22222222222222,
 'capacity': 3000.0,
 'permlanes': 2.0,
 'oneway': '1',
 'modes': {'car'},
 's2_from': 5221390723045407809,
 's2_to': 5221390723040504387,
 'attributes': {'osm:way:highway': 'SOMETHING',
  'osm:way:id': 589660342.0,
  'osm:way:lanes': '2',
  'osm:way:name': 'Shaftesbury Avenue',
  'osm:way:oneway': 'yes'},
 'length': 13.941905154249884}

In [13]:
n.change_log.head()

,timestamp,change_event,object_type,old_id,new_id,old_attributes,new_attributes,diff
0,2022-06-20 17:46:26,add,link,None,proposed_index,None,"{'from': '4356572310', 'to': '5811263955', 'id...","[(add, , [('from', '4356572310'), ('to', '5811..."
1,2022-06-20 17:46:26,add,node,None,proposed_index,None,{'data': 'some_data'},"[(add, , [('data', 'some_data')]), (add, id, p..."
2,2022-06-20 17:46:26,add,link,None,0,None,"{'from': '4356572310', 'to': '5811263955', 'id...","[(add, , [('from', '4356572310'), ('to', '5811..."
3,2022-06-20 17:46:27,modify,node,proposed_index,another_index,{'data': 'some_data'},"{'data': 'some_data', 'id': 'another_index'}","[(add, , [('id', 'another_index')]), (change, ..."
4,2022-06-20 17:46:27,modify,node,proposed_index,proposed_index,{'data': 'some_data'},"{'data': 'some_data', 'id': 'another_index'}","[(add, , [('id', 'another_index')])]"


In [14]:
n.change_log.loc[618, :]['old_attributes']

"{'id': '959', 'from': '300501141', 'to': '294158420', 'freespeed': 22.22222222222222, 'capacity': 1500.0, 'permlanes': 1.0, 'oneway': '1', 'modes': {'car'}, 's2_from': 5221390337863745429, 's2_to': 5221390337351379383, 'attributes': {'osm:relation:route': 'bus', 'osm:way:highway': 'primary', 'osm:way:id': 26785612.0, 'osm:way:name': 'Albany Street'}, 'length': 90.65828423645875}"

In [15]:
n.change_log.loc[618, :]['new_attributes']

"{'id': '959', 'from': '300501141', 'to': '294158420', 'freespeed': 22.22222222222222, 'capacity': 1500.0, 'permlanes': 1.0, 'oneway': '1', 'modes': {'car'}, 's2_from': 5221390337863745429, 's2_to': 5221390337351379383, 'attributes': {'osm:relation:route': 'bus', 'osm:way:highway': 'SOMETHING', 'osm:way:id': 26785612.0, 'osm:way:name': 'Albany Street'}, 'length': 90.65828423645875}"

In [16]:
n.change_log.loc[618, :]['diff']

[('change', 'attributes.osm:way:highway', ('primary', 'SOMETHING'))]

Another useful method is the `apply_function_to_links`/`nodes`. This function takes a user-defined function with the variable that is the data dictionary stored on the links or nodes respectively. This function does not support `conditions`, but this is something that can be encoded in you function, for example:

In [17]:
def divide_capacity(link_attributes):
    if 'car' in link_attributes['modes']:
        return link_attributes['capacity']/link_attributes['permlanes']
    
n.apply_function_to_links(divide_capacity, 'base_capacity')

2022-06-20 17:46:28,122 - Changed Link attributes for 3166 links


This method will apply results of this function to links for which a value can be computed. For use here, that means that for any link which has a `'car'` value in `'modes'` a value of capacity divided by the number of lanes will be computed. We specify that the results will be stored under `base_capacity` in the links attribute dictionary.

In [18]:
n.link(links[0])

{'id': '1007',
 'from': '4356572310',
 'to': '5811263955',
 'freespeed': 22.22222222222222,
 'capacity': 3000.0,
 'permlanes': 2.0,
 'oneway': '1',
 'modes': {'car'},
 's2_from': 5221390723045407809,
 's2_to': 5221390723040504387,
 'attributes': {'osm:way:highway': 'SOMETHING',
  'osm:way:id': 589660342.0,
  'osm:way:lanes': '2',
  'osm:way:name': 'Shaftesbury Avenue',
  'osm:way:oneway': 'yes'},
 'length': 13.941905154249884,
 'base_capacity': 1500.0}

## Subsetting

You can subset the Network object using link IDs.

In [19]:
n = read_matsim(path_to_network=network, epsg='epsg:27700', path_to_schedule=schedule, path_to_vehicles=vehicles)

links = n.extract_links_on_edge_attributes(
    conditions= {'attributes': {'osm:way:highway': ['primary', 'primary_link']}},
)

In [20]:
sub_n = n.subnetwork(links=links)

2022-06-20 17:46:30,583 - Subsetting a Network will likely result in a disconnected network graph. A cleaner will be ran that will remove links to make the resulting Network strongly connected for modes: car, walk, bike.
2022-06-20 17:46:30,776 - Param: strongly_connected_modes is defaulting to `{'car', 'walk', 'bike'}` You can change this behaviour by passing the parameter.
2022-06-20 17:46:30,779 - The graph for modes: bike does not have any connected components. This method returns True because if the graph is empty for this mode there is no reason to fail this check.
2022-06-20 17:46:30,806 - The graph for mode car is not strongly connected. The largest 1 connected components will be extracted.
2022-06-20 17:46:30,841 - Extracting largest connected components resulted in mode: car being deleted from 180 edges
/Users/kasia.kozlowska/PycharmProjects/ABM/genet/genet/core.py:591: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.loc[links & set(df.in

In [21]:
sub_n

<Network instance at 140437207025616: with 
graph: Name: 
Type: MultiDiGraph
Number of nodes: 426
Number of edges: 472
Average in degree:   1.1080
Average out degree:   1.1080 and 
schedule Schedule:
Number of services: 0
Number of routes: 0
Number of stops: 0

In [22]:
n

<Network instance at 140437209061456: with 
graph: Name: 
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 and 
schedule Schedule:
Number of services: 9
Number of routes: 68
Number of stops: 118

### Spatial Subsetting

There is a convenience method to extract a subset network using spatial conditions - refer to Using Network notebook or docs to learn more about what spatial inputs are supported.

In [23]:
# pop it in https://s2.sidewalklabs.com/ to see it
region = '48761ad0b14,48761ad0b3,48761ad0b5,48761ad0b7,48761ad0b84,48761ad0d,48761ad0e04,48761ad11f4,48761ad11fc,48761ad13,48761ad145fc,48761ad147,48761ad14c,48761ad153,48761ad41df,48761ad41e4,48761ad41fc,48761ad421,48761ad427,48761ad429,48761ad42b,48761ad5d5,48761ad5d7,48761ad5d9,48761ad5df,48761ad5e1,48761ad5e3,48761ad64,48761ad6c,48761ad71,48761ad73,48761ad744,48761ad74c,48761ad751,48761ad753,48761ad757554,48761ad75c,48761ad77,48761ad79,48761ad7b,48761ad7d,48761ad7e4,48761ad7ec,48761ad7f4,48761ad7f9,48761ad7fb,48761ad7fd,48761ad827,48761ad829,48761ad82b,48761ad9d5,48761ad9d7,48761ad9d84,48761b2802c,48761b2817,48761b281c,48761b283,48761b2847fc,48761b2849,48761b284b,48761b29b5,48761b29b7,48761b29b9,48761b29d,48761b29e4,48761b29e9,48761b29ea4,48761b29ef,48761b29fb4,48761b29fd'

In [24]:
n.subnetwork_on_spatial_condition(region, how='within')

2022-06-20 17:46:32,373 - Subsetting a Network will likely result in a disconnected network graph. A cleaner will be ran that will remove links to make the resulting Network strongly connected for modes: car, walk, bike.
2022-06-20 17:46:32,461 - Param: strongly_connected_modes is defaulting to `{'car', 'walk', 'bike'}` You can change this behaviour by passing the parameter.
2022-06-20 17:46:32,464 - The graph for modes: bike does not have any connected components. This method returns True because if the graph is empty for this mode there is no reason to fail this check.
2022-06-20 17:46:32,475 - The graph for mode car is not strongly connected. The largest 1 connected components will be extracted.
2022-06-20 17:46:32,490 - Extracting largest connected components resulted in mode: car being deleted from 72 edges
/Users/kasia.kozlowska/PycharmProjects/ABM/genet/genet/core.py:591: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df.loc[links & set(df.ind

<Network instance at 140437232395472: with 
graph: Name: 
Type: MultiDiGraph
Number of nodes: 110
Number of edges: 134
Average in degree:   1.2182
Average out degree:   1.2182 and 
schedule Schedule:
Number of services: 0
Number of routes: 0
Number of stops: 0

In [25]:
n.subnetwork_on_spatial_condition(region, how='intersect')

2022-06-20 17:46:33,820 - Subsetting a Network will likely result in a disconnected network graph. A cleaner will be ran that will remove links to make the resulting Network strongly connected for modes: car, walk, bike.
2022-06-20 17:46:33,882 - Schedule will be subsetted using given services: ['12430']. Links pertaining to their network routes will also be retained.
2022-06-20 17:46:34,443 - Removed Services with IDs `20274`, and Routes: {'VJ812fad65e7fa418645b57b446f00cba573f2cdaf', 'VJ6c64ab7b477e201cae950efde5bd0cb4e2e8888e', 'VJ375a660d47a2aa570aa20a8568012da8497ffecf'}
2022-06-20 17:46:34,642 - Removed Services with IDs `14073`, and Routes: {'VJfc35884fc4f11dc408a209c19f56f3b60f634daf', 'VJe6ba07ef9f19ae40517261ad626bf34dd656491a', 'VJea6046f64f85febf1854290fb8f76e921e3ac96b', 'VJd132b905afc6c0e8e8a994142e301ca5c0f70e22', 'VJe8cffad09738ff7b9698b333e3247918d5c45358', 'VJd9dbeefeca6d74ef2594a17514ebc08ee2d503b2', 'VJaa5ee0daec7529d7668c81fe7fac0c4ff545daea', 'VJ6cf76a4c03cca468cb

<Network instance at 140437180940944: with 
graph: Name: 
Type: MultiDiGraph
Number of nodes: 156
Number of edges: 203
Average in degree:   1.3013
Average out degree:   1.3013 and 
schedule Schedule:
Number of services: 1
Number of routes: 12
Number of stops: 83